In [ ]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import cv2

def convert_xml_to_mask(path_to_xml_data, path_to_bi_maps, target_class_name):
    # Ensure the output directory exists
    if not os.path.exists(path_to_bi_maps):
        os.makedirs(path_to_bi_maps)

    # Iterate over all XML files in the input directory
    for xml_file in os.listdir(path_to_xml_data):
        if not xml_file.endswith('.xml'):
            continue

        xml_path = os.path.join(path_to_xml_data, xml_file)
        print(f"Processing: {xml_path}")

        # Parse the XML file
        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()
        except ET.ParseError as e:
            print(f"Error parsing {xml_file}: {e}")
            continue

        # Get image dimensions
        size = root.find('size')
        if size is None:
            print(f"Skipping {xml_file}: No <size> tag found.")
            continue

        try:
            width = int(size.find('width').text.strip())
            height = int(size.find('height').text.strip())
        except (AttributeError, ValueError):
            print(f"Skipping {xml_file}: Invalid width/height.")
            continue

        # Create an empty mask image
        mask = np.zeros((height, width), dtype=np.uint8)

        # Iterate over all objects in the XML file
        for obj in root.findall('object'):
            name = obj.find('name')
            if name is None or name.text.strip() != target_class_name:
                continue  # Skip objects not matching the target class

            polygon = obj.find('polygon')
            if polygon is None:
                continue  # Skip objects without polygons

            points = []
            i = 1
            while True:
                x_tag = f"x{i}"
                y_tag = f"y{i}"

                x_elem = polygon.find(x_tag)
                y_elem = polygon.find(y_tag)

                if x_elem is None or y_elem is None:
                    break

                try:
                    x = int(float(x_elem.text.strip()))
                    y = int(float(y_elem.text.strip()))
                    points.append((x, y))
                except (AttributeError, ValueError):
                    print(f"Skipping point in {xml_file}: Invalid x/y value.")
                    continue

                i += 1

            if len(points) < 3:
                print(f"Skipping polygon in {xml_file}: Not enough points.")
                continue

            points = np.array(points, dtype=np.int32).reshape((-1, 1, 2))
            cv2.fillPoly(mask, [points], 255)

        # Save the mask image (even if it's empty – optional)
        mask_filename = os.path.splitext(xml_file)[0] + f'_{target_class_name.replace(" ", "_")}_mask.jpg'
        mask_filepath = os.path.join(path_to_bi_maps, mask_filename)
        cv2.imwrite(mask_filepath, mask)
        print(f"Saved: {mask_filepath}")

convert_xml_to_mask(
    path_to_xml_data=r"C:\Local_Docs\Uni\Hiwi_IBT\Zebra_Fish\Zebra_fish_data\Raw_data_full_train",
    path_to_bi_maps=r"C:\Local_Docs\Uni\Hiwi_IBT\Zebra_Fish\Zebra_fish_data\Raw_data_full_eyes",
    target_class_name="Front Eye" #Zebrafish or Front Eye
)


Processing: C:\Local_Docs\Uni\Hiwi_IBT\Zebra_Fish\Zebra_fish_data\Raw_data_full_train\PlatteA_119hpf_pr_1-01_jpg.rf.08bfb546c0ead6e6303928a759846143.xml
Saved: C:\Local_Docs\Uni\Hiwi_IBT\Zebra_Fish\Zebra_fish_data\Raw_data_full_eyes\PlatteA_119hpf_pr_1-01_jpg.rf.08bfb546c0ead6e6303928a759846143_Front_Eye_mask.jpg
Processing: C:\Local_Docs\Uni\Hiwi_IBT\Zebra_Fish\Zebra_fish_data\Raw_data_full_train\PlatteA_119hpf_pr_1-02_jpg.rf.b560410204d0e771da4b691a0bf0a952.xml
Saved: C:\Local_Docs\Uni\Hiwi_IBT\Zebra_Fish\Zebra_fish_data\Raw_data_full_eyes\PlatteA_119hpf_pr_1-02_jpg.rf.b560410204d0e771da4b691a0bf0a952_Front_Eye_mask.jpg
Processing: C:\Local_Docs\Uni\Hiwi_IBT\Zebra_Fish\Zebra_fish_data\Raw_data_full_train\PlatteA_119hpf_pr_1-03_jpg.rf.3697723c99ae40edceaef625f2ee6e8a.xml
Saved: C:\Local_Docs\Uni\Hiwi_IBT\Zebra_Fish\Zebra_fish_data\Raw_data_full_eyes\PlatteA_119hpf_pr_1-03_jpg.rf.3697723c99ae40edceaef625f2ee6e8a_Front_Eye_mask.jpg
Processing: C:\Local_Docs\Uni\Hiwi_IBT\Zebra_Fish\Zebra